In [67]:
# Libraries Used
import numpy as np
import pandas as pd

In [302]:
### Referenced GeeksforGeeks to read in a txt file https://www.geeksforgeeks.org/how-to-read-space-delimited-files-in-pandas/#
small_data_19 = pd.read_csv('/Users/kevintu/Documents/Python/CS205/CS_205/CS205_small_Data__19.txt', sep='  ', header=None)
### Referenced pandas library documentation for renaming a column https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
small_data_19 = small_data_19.rename(columns={0 : "label"})
print(small_data_19)

     label         1         2         3         4         5         6  \
0      1.0 -0.226035  0.390130 -1.948771  0.720117  0.406130  0.420190   
1      2.0  0.920039  0.047254  2.740796 -0.705412 -0.120161 -0.294175   
2      2.0  1.560405  1.086251 -0.497531 -0.095126  0.423804 -0.189440   
3      1.0  1.439135 -0.514825 -0.537887  0.233299  0.257673  0.102169   
4      2.0 -2.235410  1.018017  2.162868 -0.792707 -0.827919 -0.060045   
..     ...       ...       ...       ...       ...       ...       ...   
495    1.0  0.779978 -0.084679 -0.181493  1.811910 -1.170342  0.712862   
496    2.0  0.529713  1.166052 -0.449932  0.809991 -0.122535 -0.885453   
497    2.0  0.248814  0.688166 -0.839565 -1.260740 -0.780068 -0.081051   
498    2.0  0.440299  0.199763 -0.384323  0.844832  0.744562 -0.239793   
499    1.0  0.528727  0.070798 -0.589073 -0.049511 -1.395416  0.172673   

            7         8         9        10        11        12  
0   -0.646112 -1.677015 -0.719710  1.252135 -

/var/folders/6j/jnsrc7z51m96yrq49msyk2rh0000gn/T/ipykernel_13204/3780015526.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  small_data_19 = pd.read_csv('/Users/kevintu/Documents/Python/CS205/CS_205/CS205_small_Data__19.txt', sep='  ', header=None)


In [451]:
class KNN_Classifier:
    def __init__(self, train_set=None, test_set=None, k=None, nearest_neighbors=None):
        self.train_set = train_set
        self.test_set = test_set
        self.k = k
        self.nearest_neightbors = nearest_neighbors

    def distance(self, nums):
        sum = 0
        for num in nums:
            sum += num**2
        return np.sqrt(sum)
        
    def pred_class(self):
        pred_classes = []
        for row in range(self.k):
            # print("KNN: ", self.train_set[self.nearest_neighbors[row]])
            pred_classes.append(self.train_set[self.nearest_neighbors[row]])
        ### Referenced NumPy unique to return the predicted class https://numpy.org/doc/stable/reference/generated/numpy.unique.html
        classes, counts = np.unique(pred_classes, return_counts=True)
        if len(counts) == 1:
            return classes
        if counts[0] > counts[1]:
            return classes[0]
        return classes[1]
    
    def train(self):
        ### Test every testing entry to training entry
        nearest_neighbors = []
        for train_row in self.train_set:
            # print(train_row)
            # print(self.test_set)
            # for test_row in self.test_set:
            #     print(test_row)
            nearest_neighbors.append(self.distance(train_row - self.test_set))
        # print(nearest_neighbors)
        ### Referenced NumPy argsort to sort by index of distances https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
        self.nearest_neighbors = np.argsort(nearest_neighbors)
        # print(self.nearest_neighbors)


In [452]:
### Start by splitting my data into train and test set
data = {'label': [2.0000000e+000, 1.0000000e+000, 2.0000000e+000, 1.0000000e+000, 2],
        'feature_1': [1.2340000e+010, 6.0668000e+000, 2.3400000e+010, 4.5645400e+010, 1],
        'feature_2': [ 2.3440000e+000, 5.0770000e+000, 3.6460000e+000, 3.0045000e+000, 2]}
data = {'label': [2, 1, 2, 1, 2],
        'feature_1': [1, 10, 2, 11, 1],
        'feature_2': [ 2, 11, 3, 12, 2]}
df = pd.DataFrame(data)
# print(df)
train_data = df.sample(frac = 0.8, random_state = 4)
train_data.reset_index(drop=True, inplace=True)
print(train_data)
test_data = df.drop(train_data.index)
test_data.reset_index(drop=True, inplace=True)
print(test_data)

knn = KNN_Classifier(train_data.to_numpy(), test_data.to_numpy(), 3)
knn.train()
print(knn.pred_class())

   label  feature_1  feature_2
0      2          1          2
1      1         11         12
2      1         10         11
3      2          1          2
   label  feature_1  feature_2
0      2          1          2
1


In [455]:
class Feature_Selection:
    def __init__(self, dataset=None, k=None, feature_set=None):
        self.dataset=dataset
        self.k=k
        self.feature_set = []
    
    def print_feature_set(self):
        print(self.feature_set)
        
    def greedy_forward(self):
        if self.dataset.shape[1] == 1:
            return
        precision = []
        ### Retreiving the Columns for Feature Selection
        for column in self.dataset.columns:
            if column == 'label':
                continue
            # print(column)
            feature = self.dataset[['label', column]].to_numpy()
            # print(feature)
            correct = 0
            ### K-Fold CV
            for cfv in range(int(len(feature)/self.k)):
                # print(cfv)
                # print(len(feature)/self.k)
                train_data = np.delete(feature, cfv, axis=0)
                # print(train_data)
                test_data = feature[cfv]
                # print(test_data)

                knn = KNN_Classifier(train_data, test_data, 1)
                knn.train()
                # print(test_data[0])
                # print(knn.pred_class())
                if knn.pred_class() == test_data[0]:
                    # print('HELLO')
                    correct += 1
            precision.append(correct/int(len(feature)/self.k))
        print(precision)
        print("Best feature performed with: ", precision[-1])
        precision = np.argsort(precision)
        print(precision)
        print("Selected feature is: ",self.dataset.columns[precision[-1]]+1)
        self.feature_set.append(self.dataset.columns[precision[-1]])
        print(self.feature_set)
        ### Referenced Pandas library to understand how to drop column https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
        self.dataset = self.dataset.drop(columns=[self.dataset.columns[precision[-1]]])
        # print(self.dataset)
        # self.greedy_forward()
        self.print_feature_set
        pass
    
    def greedy_backward(self):
        pass

In [456]:
### Start by splitting my data into train and test set
data = {'label': [2, 1, 2, 1, 2],
        'feature_1': [1, 10, 2, 11, 1],
        'feature_2': [ 2, 11, 3, 12, 2]}
df = pd.DataFrame(data)
# print(df)
df = small_data_19
train_data = df.sample(frac = 0.8, random_state = 4)
train_data.reset_index(drop=True, inplace=True)
print(train_data)
test_data = df.drop(train_data.index)
test_data.reset_index(drop=True, inplace=True)
print(test_data)

feat_select = Feature_Selection(train_data, 1)
feat_select.greedy_forward()
feat_select.greedy_backward()


     label         1         2         3         4         5         6  \
0      1.0 -1.444748  0.240445  1.163676  1.366546 -1.163196  0.074090   
1      2.0 -0.810568  0.601724  0.393059 -1.788170  1.214824 -0.497572   
2      2.0  1.008960 -1.023832  0.311831  1.965024  0.113587  1.501350   
3      1.0  0.717740 -0.236836  0.401796 -1.378916  0.445133  0.719735   
4      2.0  0.912406 -0.602166 -0.496746 -0.912864 -0.120059 -0.138839   
..     ...       ...       ...       ...       ...       ...       ...   
395    2.0 -1.208482  0.800707 -0.844830 -0.572029  0.894568 -1.397561   
396    1.0 -0.039568  0.823157  0.913989  0.556955 -0.361448  0.826168   
397    2.0  0.628419  0.612144  2.282201  1.549427 -0.593250 -0.446338   
398    2.0  1.310620  0.965883 -1.349171 -0.814235  0.198841 -2.387644   
399    2.0  0.277867 -0.262804 -0.256040  2.306253 -0.118172 -0.108429   

            7         8         9        10        11        12  
0   -0.541628  0.185851 -0.155851 -0.471252  